In [1]:
import random as rd
import numpy as np
import pandas as pd
import os

from scoring import Metrics
from classification import Random

In [2]:
data = np.array([
    [28, 3, 10, 0],
    [48, 1, 23, 1],
    [42, 10, 12, 1],
    [32, 11, 7, 1],
    [1, 7, 123, 0],
    [34, 2, 17, 1],
    [35, 3, 100, 0],
    [13, 12, 12, 0],
    [83, 2, 3, 1],
    [39, 12, 88, 1],
    [128, 23, 33, 0]
])





# sel = 10
# index = np.random.choice(data.shape[0], sel, replace=False)
# print(index)
# data[index]

In [3]:
DIRPATH = 'src'

# Import data
import_path = os.path.join(DIRPATH, 'export.npy')

data = np.load (import_path)
#data[:, :-1] = data[:, :-1]/255.  # normalise values
vals_nb = data.shape[0]

In [4]:
# Create train and test sets (19.2s in low power, 11.5s)

"""
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)
"""

def create_sets(data, train_frac=0.05):

    # Create (train, test)
    data_shuffled = data.copy()

    np.random.seed(0)
    np.random.shuffle(data_shuffled)

    train_nb = int(vals_nb * train_frac)
    # print(f'training on {train_nb} vals')

    return data_shuffled[:train_nb], data_shuffled[train_nb:]

data_train, data_test = create_sets(data, train_frac=0.02)  # changed from 0.3

In [5]:
import cv2

# Test image
image_test_path = 'src/ima_1e8ccf23-d106-4227-908c-e4fbbb1da5f8.jpg'

image_test = image_test_path.split('/')[-1]
image_test_name, _ = os.path.splitext(image_test)
id_test = image_test_name[4:]

# Import and flatten test image
image_test_path = os.path.join('src/sky-images', image_test)
image_test_array = cv2.imread (image_test_path)
image_test_shape = image_test_array.shape
image_test_array_flat = image_test_array.reshape(-1, 1, 3)# / 255.
image_test_array_flat = image_test_array_flat.squeeze()

In [6]:
from classifiers.kernel import Kernel_bin

RUN_KERNEL = True
if RUN_KERNEL:
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test,  y_test  = data_test [:, :-1], data_test [:, -1]

    # Train kernel
    kernel = Kernel_bin(func = None)
    kernel.fit(X_train, y_train)

    # Test kernel
    y_pred = kernel.predict(X_test, batch_size = 10000)
    metrics_kernel = Metrics(y_pred, y_test)
    print(metrics_kernel)

    # Test on test image
    image_test_pred = kernel.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-kernel.png', image_test_pred * 255)

"""
Eval time = 33.5s (low power)
Batch size = 10000
=================================
TP  = 3251481
FP  = 248305
FN  = 1018523
TN  = 5211397
P   = 4270004
N   = 5459702
=================================
TPR = 0.7614702468662793
TNR = 0.9545204115535976
FPR = 0.04547958844640239
FNR = 0.2385297531337207
=================================
Precision           = 0.9290513762841499
False omission rate = 0.04547958844640239
Error rate          = 0.13020208421508317
Accuracy            = 0.8697979157849168
=================================
F1-score            = 0.8369546667284444
MCC                 = 0.7403617729057405
Kappa score         = 0.7303439199164562
""";

(9729706, 3)
TP  = 3251481
FP  = 248305
FN  = 1018523
TN  = 5211397
P   = 4270004
N   = 5459702
TPR = 0.7614702468662793
TNR = 0.9545204115535976
FPR = 0.04547958844640239
FNR = 0.2385297531337207
Precision           = 0.9290513762841499
False omission rate = 0.04547958844640239
Error rate          = 0.13020208421508317
Accuracy            = 0.8697979157849168
F1-score            = 0.8369546667284444
MCC                 = 0.7403617729057405
Kappa score         = 0.7303439199164562


In [ ]:
from classifiers.lda import LDA_bin as LDA

RUN_LDA = False
if RUN_LDA:
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test,  y_test  = data_test [:, :-1], data_test [:, -1]

    # Train LDA
    lda = LDA()
    lda.fit (X_train, y_train)

    # Test LDA
    y_pred = lda.predict(X_test)
    metrics_lda = Metrics(y_pred, y_test)
    print(metrics_lda)

    # Test on test image
    image_test_pred = lda.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-lda.png', image_test_pred * 255)

"""
Eval time = 0.4s
=================================
TP  = 2273658
FP  = 225912
FN  = 328428
TN  = 4121792
P   = 2602086
N   = 4347704
=================================
TPR = 0.8737828034892006
TNR = 0.9480387809289685
FPR = 0.05196121907103152
FNR = 0.1262171965107994
=================================
Precision           = 0.9096196545805879
False omission rate = 0.05196121907103152
Error rate          = 0.07976356120112982
Accuracy            = 0.9202364387988702
=================================
F1-score            = 0.8913411645159924
MCC                 = 0.8287908705201819
Kappa score         = 0.8283733047108975
""";

In [6]:
### QDA
from classifiers.qda import QDA_bin as QDA

RUN_QDA = False
if RUN_QDA:
    X_train, y_train = data_train[:, :-1], data_train[:, -1]
    X_test,  y_test  = data_test [:, :-1], data_test [:, -1]

    # Train QDA
    qda = QDA()
    qda.fit (X_train, y_train)

    # Test QDA
    y_pred = qda.predict(X_test)
    metrics_qda = Metrics(y_pred, y_test)
    print(metrics_qda)

    # Test on test image
    image_test_pred = qda.predict(image_test_array_flat)
    image_test_pred = image_test_pred.reshape(image_test_shape[:2])
    cv2.imwrite(f'output/{image_test_name}-pred-qda.png', image_test_pred * 255)

"""
Eval time = 0.4s
=================================
TP  = 2441037
FP  = 58533
FN  = 658013
TN  = 3792207
P   = 3099050
N   = 3850740
=================================
TPR = 0.7876726738839321
TNR = 0.9847995450225151
FPR = 0.015200454977484847
FNR = 0.21232732611606783
=================================
Precision           = 0.9765827722368248
False omission rate = 0.015200454977484847
Error rate          = 0.10310325923517114
Accuracy            = 0.8968967407648288
=================================
F1-score            = 0.8720138176907881
MCC                 = 0.8001029410421224
Kappa score         = 0.7873374479219594
""";

In [9]:
# ### TODO: k-NN


# from classifiers.trees import KDTree, DecisionTree

# data = np.array([
#     [28, 3, 10, 0],
#     [48, 1, 23, 1],
#     [42, 10, 12, 1],
#     [32, 11, 7, 1],
#     [1, 7, 123, 0],
#     [34, 2, 17, 1],
#     [35, 3, 100, 0],
#     [13, 12, 12, 0],
#     [83, 2, 3, 1],
#     [39, 12, 88, 1],
#     [128, 23, 33, 0]
# ])


# tree = KDTree (data, dimension=3)
# tree.grow()
# print(tree)



# # dtree = DecisionTree(data)
# # dtree.grow()
# # print(dtree)